(doc, term) matrix 를 x0 로 읽어들인 뒤, 이를 bipartite graph 인 x 로 만듭니다. 그래프에서 `단어 0 번`과 `문서 0 번`은 서로 다른 마디입니다. bow_to_graph 함수를 이용하여 sparse matrix 로부터 그래프를 만듭니다. Row 와 column 이 각각 그래프의 마디가 됩니다.

그래프의 마디 인덱스는 [Row | Column] 으로 만들어졌습니다. x 에서 0 부터 30090 까지는 문서이며 30091 부터 39864 까지는 단어입니다.

In [1]:
import config
import soygraph
from soygraph import bow_to_graph
from navernews_10days import get_bow

x0, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
x = bow_to_graph(x0)

soynlp=0.0.491
added lovit_textmining_dataset


Random Walk with Restart (RWR) 함수를 import 한 뒤, graph 를 입력합니다.

In [2]:
from soygraph.similarity import RandomWalkWithRestart

# Random walk with Restart 를 만듭니다. 
# graph 를 입력합니다. 
rwr = RandomWalkWithRestart(x)

`아이오아이`의 index 는 5537 입니다.

In [3]:
# query word 의 index 를 찾습니다.
vocab_to_idx['아이오아이']

5537

문서의 개수에 단어 index 를 더하여 seed node 를 입력합니다. Seed node 로부터 가까운 topk 개의 마디를 찾습니다. Random Walk 는 6 steps 를 움직입니다. Topically similar terms 가 탐색됨을 알 수 있습니다.

In [5]:
%%time
# 문서 길이 + vocabulary idx
similars_rwr = rwr.most_similar(x0.shape[0] + 5537, topk=30, max_iter=6)

for idx, _ in similars_rwr:
    print(idx_to_vocab[idx - x0.shape[0]])

빅브레인
너무너무너무
오블리스
신용재
갓세븐
아이오아이
엠카운트다운
다비치
세븐
완전체
박진영
펜타곤
산들
중독성
엠넷
열창
잠깐
깜찍
타이틀곡
상큼
소녀들
몬스타엑스
일산동구
키미
불독
프로듀스
소라
방탄소년단
형은
파워풀
CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 80.8 ms


Single Vector SimRank 는 sparse matrix 가 아닌 Python dict 로 구현하였습니다. 이를 이용하기 위해 sparse matrix 를 dict 로 변환하는 matrix_to_dict 함수를 불러들입니다.

In [6]:
from soygraph import matrix_to_dict

# sparse matrix 를 dict - dict 로 변환합니다.
dd = matrix_to_dict(x)
print(len(dd))

39776


dict 를 DictGraph 로 만듭니다.

In [7]:
from soygraph import DictGraph

# soygraph 의 DictGraph 를 이용하여 graph 를 만듭니다.
g = DictGraph(dd)

Single Vector SimRank 에 DictGraph g 를 입력합니다.

In [8]:
from soygraph.similarity import SingleVectorSimRank

# Single Vector SimRank 를 만듭니다. 
# graph 를 입력합니다. 
simrank = SingleVectorSimRank(g)

동일한 작업을 수행합니다. RWR 은 빠른 시간에 계산이 되지만, Single Vector SimRank 는 좀 느립니다. 이는 Python dict 를 이용한, 최적화된 구현체가 아니기 때문이기도 하지만, SimRank 는 Random Walk with Restart 보다 더 많은 과정을 거쳐 계산이 되기 때문에 본래 구현체가 좋아도 느립니다. 하지만 topically similar terms 가 찾아짐은 동일합니다. Bipartite graph 임을 알고 있다면, SimRank 보다는 RWR 에서 random walk steps 를 짝수 번 수행하여 topically similarterms 를 탐색하는 것이 더 효율적입니다.

In [9]:
%%time
similars_simrank = simrank.most_similar(x0.shape[0] + 5537, max_iter=4, topk=30)
for idx, _ in similars_simrank:
    print(idx_to_vocab[idx - x0.shape[0]])

너무너무너무
박진영
빅브레인
완전체
신용재
오블리스
갓세븐
엠카운트다운
중독성
잠깐
세븐
다비치
상큼
소녀들
선의
산들
수록곡
프로듀스101
펜타곤
열창
타이틀곡
엠넷
본명
박소라
음원차트
깜찍
이진희
불독
키미
음악방송
CPU times: user 15.9 s, sys: 4 ms, total: 15.9 s
Wall time: 15.9 s
